In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


##Import

In [2]:
import pandas
import numexpr
import numpy
from rep_ef.estimators import MatrixNetSkyGridClassifier
from rep.metaml import FoldingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score

In [3]:
%run utils.py

## Reading initial data

In [4]:
import root_numpy
data = pandas.DataFrame(root_numpy.root2array('datasets/nnet_vtx.root'))

In [4]:
# import root_numpy
# data = pandas.DataFrame(root_numpy.root2array('datasets/vtx1.root'))

## Define label
`label` = `signB` * `signVtx`
* the highest output means that this is same sign B as vtx (right tagged) - label 1
* the lowest output means that this is opposite sign B than vtx (wrong tagged) - label 0

In [5]:
data['label'] = data.iscorrect
data['event'] = data.evtNum 

In [6]:
data.head()

,iscorrect,tagger,evtNum,runNum,nnkrec,mult,ptB,etaB,partP,partPt,...,BDphiDir,svp,om_muon,om_ele,om_kaon,om_same,om_vtx,N_sig_sw,label,event
0,1,0,490856995,115839,3,19,-0.382002,-6.307031,1.673492,0.032547,...,-1.014496,5.458805,0,0,0,0,0,1.034008,1,490856995
1,1,0,68238381,115839,4,16,0.788837,-4.426422,2.556391,0.068375,...,-1.098808,4.479315,0,0,0,0,0,-0.263734,1,68238381
2,0,0,3537659,115839,3,38,0.070327,-4.690033,3.843373,0.622620,...,-0.011526,4.092348,0,0,0,0,0,1.062726,0,3537659
3,0,0,828473522,115839,3,51,0.206689,-5.277009,1.386402,-0.496672,...,-0.151818,2.874062,0,0,0,0,0,0.249383,0,828473522
4,0,0,186799080,115839,3,45,1.195913,-4.579178,2.385584,-0.135151,...,-2.699381,2.975542,0,0,0,0,0,0.981713,0,186799080


In [7]:
features = ['mult', 'nnkrec', 'ptB', 'vflag', 'ipsmean', 'ptmean', 'vcharge', 
            'svm', 'svp', 'BDphiDir', 'svtau', 'docamax']

In [8]:
statistics(data)

{'Events': 201421, 'tracks': 201431}

In [9]:
N_all = 742632.88650423975 

In [10]:
N_pass = sum([gr['N_sig_sw'].values[0] for _, gr in data.groupby('event')])
eff_tag = 1. * N_pass / N_all
eff_delta = sqrt(N_pass) / N_all
eff_tag, eff_delta

(0.18203126835163155, 0.00049509199905288309)

### define B-like events for training and others for prediction

In [11]:
sweight_threshold = 0.
data_sw_pass = data[data.N_sig_sw > sweight_threshold]
data_sw_not_pass = data[data.N_sig_sw <= sweight_threshold]
statistics(data_sw_pass)

{'Events': 157233, 'tracks': 157236}

### MN full

In [12]:
from rep_ef.estimators import MatrixNetSkyGridClassifier
mn_base = MatrixNetSkyGridClassifier(connection='skygrid', user_name='antares',
                                     iterations=3000, regularization=0.02, sync=False)

mn_folding = FoldingClassifier(mn_base, n_folds=2, random_state=11, 
                               features=features)
mn_folding.fit(data_sw_pass, data_sw_pass.label, data_sw_pass.N_sig_sw)

FoldingClassifier(base_estimator=MatrixNetSkyGridClassifier(auto_stop=None, baseline_feature=None,
              command_line_params=None, connection='skygrid',
              dump_filename=None, features_sample_rate_per_iteration=1.0,
              intervals=64, iterations=3000, max_features_per_iteration=6,
              regularization=0.02, sync=False, train_features=None,
              training_fraction=0.5, user_name='antares'),
         features=['mult', 'nnkrec', 'ptB', 'vflag', 'ipsmean', 'ptmean', 'vcharge', 'svm', 'svp', 'BDphiDir', 'svtau', 'docamax'],
         ipc_profile=None, n_folds=2, random_state=11)

In [13]:
probs_RT = mn_folding.predict_proba(data_sw_pass)[:, 1]
roc_auc_score(data_sw_pass.label.values, probs_RT, sample_weight=data_sw_pass.N_sig_sw.values)

KFold prediction using folds column


0.56817442178726196

### XGB full

In [14]:
from rep.estimators import XGBoostClassifier
xgb_base = XGBoostClassifier(colsample=1.0, eta=0.01, nthreads=4, 
                             n_estimators=500, subsample=0.3, max_depth=3) 
xgb_folding = FoldingClassifier(xgb_base, n_folds=2, random_state=11, 
                                features=features)
xgb_folding.fit(data_sw_pass, data_sw_pass.label, data_sw_pass.N_sig_sw)

FoldingClassifier(base_estimator=XGBoostClassifier(base_score=0.5, colsample=1.0, eta=0.01, features=None,
         gamma=None, max_depth=3, min_child_weight=1.0, missing=-999.0,
         n_estimators=500, nthreads=4, num_feature=None, random_state=0,
         scale_pos_weight=1.0, subsample=0.3, verbose=0),
         features=['mult', 'nnkrec', 'ptB', 'vflag', 'ipsmean', 'ptmean', 'vcharge', 'svm', 'svp', 'BDphiDir', 'svtau', 'docamax'],
         ipc_profile=None, n_folds=2, random_state=11)

In [15]:
probs_RT = xgb_folding.predict_proba(data_sw_pass)[:, 1]
roc_auc_score(data_sw_pass.label.values, probs_RT, sample_weight=data_sw_pass.N_sig_sw.values)

KFold prediction using folds column


0.56895875930786133

--------

## Isotonic calibration to probabity p(vrt same sign|B)

In [16]:
models = []

### MN

In [17]:
D2, alpha, aucs, iso_calibs = calibration(mn_folding, 
                                          [data_sw_pass, data_sw_not_pass], 
                                          steps=30)

KFold prediction using folds column
KFold prediction using folds column


In [18]:
print 'AUC', numpy.mean(aucs), numpy.var(aucs)

AUC 0.571638856332 2.70392543927e-06


In [19]:
models.append(result_table(eff_tag, eff_delta, D2, 'mn'))

### XGB

In [20]:
D2, alpha, aucs, iso_calibs = calibration(xgb_folding, 
                                          [data_sw_pass, data_sw_not_pass], 
                                          steps=30)

KFold prediction using folds column
KFold prediction using folds column


In [21]:
print 'AUC', numpy.mean(aucs), numpy.var(aucs)

AUC 0.572830247879 2.68774957026e-06


In [22]:
models.append(result_table(eff_tag, eff_delta, D2, 'xgb'))

In [23]:
pandas.concat(models)

,name,"$\epsilon_{tag}, \%$","$\Delta \epsilon_{tag}, \%$",$D^2$,$\Delta D^2$,"$\epsilon, \%$","$\Delta \epsilon, \%$"
0,mn,18.203127,0.049509,0.051586,0.000001,0.939027,0.002554
0,xgb,18.203127,0.049509,0.050944,0.000001,0.927345,0.002522
